In [ ]:
import json
from elasticsearch import Elasticsearch

# Connect to Elasticsearch
es = Elasticsearch(
    "https://localhost:9200",
    ca_certs=".\certs\ca\ca.crt",
    basic_auth=("elastic", "hammasir"),
    verify_certs=False,
    ssl_show_warn=False)
es.info()

In [3]:
with open('../data/processed/base_dataset.json') as f:
    json_data = json.load(f)


In [12]:
import pandas as pd
data = pd.read_csv("../data/processed/base_dataset.csv")

In [7]:
import numpy as np
data = data.replace(np.nan, None)

In [21]:
mappings = {
        "properties": {
            "gender": {"type": "text"},
            "expertise": {"type": "text"},
            "title": {"type": "text"},
            "star": {"type": "float"},
            "rates_count": {"type": "integer"},
            "number_of_visits": {"type": "integer"},
            "view": {"type": "text"},
            "insurances": {"type": "text"},
            "experience": {"type": "integer"},
            "rate_info": {"type": "object",
                          "properties": {
                              "rates_count": "integer",
                              "doctor_encounter": "float",
                              "explanation_of_issue": "float",
                              "waiting_time": "float",
                              "comments_count": "integer",
                            #   "waiting_time_count": "text",
                              "rate": "float"
                          }
                          },
            "clinic": {"type": "nested"},
    }
}

In [ ]:
es.indices.delete(index="doctors")

In [ ]:
es.indices.create(index="doctors", mappings=mappings)

In [ ]:
for i, row in data.iterrows():
    doc = {
        "expertise": row["display_expertise"],
        "gender": row["gender"],
        "experience": row["experience"],
        "title": row["title"],
        "star": row["star"],
        "rates_count" : row["rates_count"],
        "number_of_visits": row["number_of_visits"],
        "view": row["view"],
        "insurances": row["insurances"],
        "rate_info": row["rate_info"],
        "clinic": row["clinic"]
    }

    es.index(index="doctors", id=i, document=doc)

In [34]:
es.indices.refresh(index="doctors")
es.cat.count(index="doctors", format="json")

ListApiResponse([{'epoch': '1725205909', 'timestamp': '15:51:49', 'count': '16296'}])

In [42]:
es.search(index="doctors", q="ماما")

ObjectApiResponse({'took': 7, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 13.364021, 'hits': [{'_index': 'doctors', '_id': '10820', '_score': 13.364021, '_source': {'expertise': 'ماما', 'gender': 'F', 'experience': None, 'title': 'محبوبه رستگارراد', 'star': 5.0, 'rates_count': 1.0, 'number_of_visits': 1035.0, 'view': '1K', 'insurances': '[]', 'rate_info': "{'quality_of_treatment': 5, 'waiting_time_count': 1, 'waiting_time': 0, 'rate': 5, 'comments_count': 1, 'rates_count': 1, 'doctor_encounter': 5, 'explanation_of_issue': 5, 'count_dislike': 0}", 'clinic': "[{'city': 'مشهد', 'number': None, 'address': 'مشهد، بلوار حر، بین حر 7  و 9', 'province_name': 'خراسان رضوی'}]"}}, {'_index': 'doctors', '_id': '10758', '_score': 9.523348, '_ignored': ['clinic.keyword'], '_source': {'expertise': 'زنان _ مامایی _ ماما همراه _ ناباروری', 'gender': 'F', 'experience': None, 'title': 'صابری مفرد',